## Preprocessing of Loan Overdue Dataset

### Introduction

We use the dataset of [Tianchi Competetion](https://tianchi.aliyun.com/competition/entrance/531830/information) to train our loan overdue rate estimation. In this notebook, we preprocess the dataset and generate features, which refers to some execellent work listed as below:

### References
* **Overview**: https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.6.3b30250fXUZ5fy&postId=129318
* **EDA**: https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.12.3b30250fXUZ5fy&postId=129320
* **Feature Eningeering**: https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.6.3b30b135z4zdwX&postId=129321
* **Hypermeter Tunning**: https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.3.3b30b1352BkwCe&postId=129346
* **Stacking and Blending**: https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12586969.1002.15.3b30b1352BkwCe&postId=131986

In [1]:
import pandas as pd
import numpy as np
import datetime
import warnings
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

### Read data

To download the dataset to your own s3 bucket:

* Fill {YOUR_S3_BUCKET} and {YOUR_S3_PATH} with your preferred values in the following cell.
* Uncomment the cell by removing the leading # character.
* Execute the cell.

In [2]:
# !aws s3 cp ${MY_S3_BUCKET}/risk/tianchi/train.csv .
# !aws s3 cp ${MY_S3_BUCKET}/risk/tianchi/testA.csv .

In [3]:
data_train = pd.read_csv('./train.csv')
data_test_a = pd.read_csv('./testA.csv')

In [4]:
# numerical features and categorical features
numerical_fea = list(data_train.select_dtypes(exclude=['object']).columns)
category_fea = list(filter(lambda x: x not in numerical_fea, list(data_train.columns)))
label = 'isDefault'
numerical_fea.remove(label)

In [5]:
numerical_fea

['id',
 'loanAmnt',
 'term',
 'interestRate',
 'installment',
 'employmentTitle',
 'homeOwnership',
 'annualIncome',
 'verificationStatus',
 'purpose',
 'postCode',
 'regionCode',
 'dti',
 'delinquency_2years',
 'ficoRangeLow',
 'ficoRangeHigh',
 'openAcc',
 'pubRec',
 'pubRecBankruptcies',
 'revolBal',
 'revolUtil',
 'totalAcc',
 'initialListStatus',
 'applicationType',
 'title',
 'policyCode',
 'n0',
 'n1',
 'n2',
 'n3',
 'n4',
 'n5',
 'n6',
 'n7',
 'n8',
 'n9',
 'n10',
 'n11',
 'n12',
 'n13',
 'n14']

In [6]:
category_fea

['grade', 'subGrade', 'employmentLength', 'issueDate', 'earliesCreditLine']

### Fill null values for numerical and categorical features seperately

In [7]:
data_train[numerical_fea].isnull().sum()

id                        0
loanAmnt                  0
term                      0
interestRate              0
installment               0
employmentTitle           1
homeOwnership             0
annualIncome              0
verificationStatus        0
purpose                   0
postCode                  1
regionCode                0
dti                     239
delinquency_2years        0
ficoRangeLow              0
ficoRangeHigh             0
openAcc                   0
pubRec                    0
pubRecBankruptcies      405
revolBal                  0
revolUtil               531
totalAcc                  0
initialListStatus         0
applicationType           0
title                     1
policyCode                0
n0                    40270
n1                    40270
n2                    40270
n3                    40270
n4                    33239
n5                    40270
n6                    40270
n7                    40270
n8                    40271
n9                  

In [8]:
data_train[category_fea].isnull().sum()

grade                    0
subGrade                 0
employmentLength     46799
issueDate                0
earliesCreditLine        0
dtype: int64

In [9]:
# for numerical features we use median values
data_train[numerical_fea] = data_train[numerical_fea].fillna(data_train[numerical_fea].median())
data_test_a[numerical_fea] = data_test_a[numerical_fea].fillna(data_train[numerical_fea].median())

# for categorical features we use mode values
data_train[category_fea] = data_train[category_fea].fillna(data_train[category_fea].mode())
data_test_a[category_fea] = data_test_a[category_fea].fillna(data_train[category_fea].mode())

In [10]:
data_train.isnull().sum()

id                        0
loanAmnt                  0
term                      0
interestRate              0
installment               0
grade                     0
subGrade                  0
employmentTitle           0
employmentLength      46799
homeOwnership             0
annualIncome              0
verificationStatus        0
issueDate                 0
isDefault                 0
purpose                   0
postCode                  0
regionCode                0
dti                       0
delinquency_2years        0
ficoRangeLow              0
ficoRangeHigh             0
openAcc                   0
pubRec                    0
pubRecBankruptcies        0
revolBal                  0
revolUtil                 0
totalAcc                  0
initialListStatus         0
applicationType           0
earliesCreditLine         0
title                     0
policyCode                0
n0                        0
n1                        0
n2                        0
n3                  

In [11]:
# ffill for null values `employmentLength`
data_train = data_train.fillna(axis=0, method='ffill')
data_test_a = data_test_a.fillna(axis=0, method='ffill')

In [12]:
data_train.isnull().sum()

id                    0
loanAmnt              0
term                  0
interestRate          0
installment           0
grade                 0
subGrade              0
employmentTitle       0
employmentLength      0
homeOwnership         0
annualIncome          0
verificationStatus    0
issueDate             0
isDefault             0
purpose               0
postCode              0
regionCode            0
dti                   0
delinquency_2years    0
ficoRangeLow          0
ficoRangeHigh         0
openAcc               0
pubRec                0
pubRecBankruptcies    0
revolBal              0
revolUtil             0
totalAcc              0
initialListStatus     0
applicationType       0
earliesCreditLine     0
title                 0
policyCode            0
n0                    0
n1                    0
n2                    0
n3                    0
n4                    0
n5                    0
n6                    0
n7                    0
n8                    0
n9              

### Transform `issueDate` into numerical values

In [13]:
for data in [data_train, data_test_a]:
    data['issueDate'] = pd.to_datetime(data['issueDate'],format='%Y-%m-%d')
    startdate = datetime.datetime.strptime('2007-06-01', '%Y-%m-%d')
    data['issueDateDT'] = data['issueDate'].apply(lambda x: x-startdate).dt.days

In [14]:
data[:3]

,id,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,issueDate,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issueDateDT
0,800000,14000.0,3,10.99,458.28,B,B3,7027.0,10+ years,0,80000.0,0,2014-07-01,0,163.0,21,10.56,1.0,715.0,719.0,17.0,0.0,0.0,9846.0,30.7,29.0,0,0,Nov-1974,0.0,1.0,1.0,4.0,6.0,6.0,6.0,8.0,4.0,15.0,19.0,6.0,17.0,0.0,0.0,1.0,3.0,2587
1,800001,20000.0,5,14.65,472.14,C,C5,60426.0,10+ years,0,50000.0,0,2015-07-01,2,235.0,8,21.40,2.0,670.0,674.0,5.0,0.0,0.0,8946.0,56.6,14.0,0,0,Jul-2001,5.0,1.0,2.0,1.0,3.0,3.0,1.0,1.0,3.0,3.0,9.0,3.0,5.0,0.0,0.0,2.0,2.0,2952
2,800002,12000.0,3,19.99,445.91,D,D4,23547.0,2 years,1,60000.0,2,2016-10-01,0,526.0,20,33.50,0.0,710.0,714.0,12.0,0.0,0.0,970.0,17.6,43.0,1,0,Aug-2006,0.0,1.0,0.0,1.0,4.0,4.0,1.0,1.0,36.0,5.0,6.0,4.0,12.0,0.0,0.0,0.0,7.0,3410


### Transform `employmentLength` into numerical values

In [15]:
data_train['employmentLength'].value_counts(dropna=False).sort_index()

1 year        55842
10+ years    278860
2 years       76742
3 years       68149
4 years       50932
5 years       53186
6 years       39575
7 years       37622
8 years       38551
9 years       32225
< 1 year      68316
Name: employmentLength, dtype: int64

In [16]:
def employmentLength_to_int(s):
    if pd.isnull(s):
        return s
    else:
        return np.int8(s.split()[0])
for data in [data_train, data_test_a]:
    data['employmentLength'].replace(to_replace='10+ years', value='10 years', inplace=True)
    data['employmentLength'].replace('< 1 year', '0 years', inplace=True)
    data['employmentLength'] = data['employmentLength'].apply(employmentLength_to_int)

data['employmentLength'].value_counts(dropna=False).sort_index()

0     16990
1     14017
2     19347
3     17012
4     12566
5     13324
6      9898
7      9304
8      9595
9      8073
10    69874
Name: employmentLength, dtype: int64

### Transform `earliesCreditLine` into numerical values

In [17]:
data_train['earliesCreditLine'].sample(5)

411886    Dec-2009
69250     Dec-1993
597454    Jul-2004
717143    Jun-1999
4910      Oct-1992
Name: earliesCreditLine, dtype: object

In [18]:
# tranform earliesCreditLine into numerical values
for data in [data_train, data_test_a]:
    data['earliesCreditLine'] = data['earliesCreditLine'].apply(lambda s: int(s[-4:]))

### Encode the categorical features

In [19]:
cate_features = ['grade', 'subGrade', 'employmentTitle', 'homeOwnership', 'verificationStatus', 'purpose', 'postCode', 'regionCode', \
                 'applicationType', 'initialListStatus', 'title', 'policyCode']
for f in cate_features:
    print(f, 'different values：', data[f].nunique())

grade different values： 7
subGrade different values： 35
employmentTitle different values： 79282
homeOwnership different values： 6
verificationStatus different values： 3
purpose different values： 14
postCode different values： 889
regionCode different values： 51
applicationType different values： 2
initialListStatus different values： 2
title different values： 12058
policyCode different values： 1


In [20]:
for data in [data_train, data_test_a]:
    data['grade'] = data['grade'].map({'A':1,'B':2,'C':3,'D':4,'E':5,'F':6,'G':7})

In [21]:
for data in [data_train, data_test_a]:
    data = pd.get_dummies(data, columns=['subGrade', 'homeOwnership', 'verificationStatus', 'purpose', 'regionCode'], drop_first=True)

### Outliers processing

In [22]:
def find_outliers_by_3segama(data,fea):
    data_std = np.std(data[fea])
    data_mean = np.mean(data[fea])
    outliers_cut_off = data_std * 3
    lower_rule = data_mean - outliers_cut_off
    upper_rule = data_mean + outliers_cut_off
    data[fea + '_outliers'] = data[fea].apply(lambda x:str('ExceptionValue') if x > upper_rule or x < lower_rule else 'NormalValue')
    return data

In [23]:
for fea in numerical_fea:
    data_train = find_outliers_by_3segama(data_train,fea)
    print(data_train[fea + '_outliers'].value_counts())
    print(data_train.groupby(fea + '_outliers')['isDefault'].sum())
    print('-' * 60)

NormalValue    800000
Name: id_outliers, dtype: int64
id_outliers
NormalValue    159610
Name: isDefault, dtype: int64
------------------------------------------------------------
NormalValue    800000
Name: loanAmnt_outliers, dtype: int64
loanAmnt_outliers
NormalValue    159610
Name: isDefault, dtype: int64
------------------------------------------------------------
NormalValue    800000
Name: term_outliers, dtype: int64
term_outliers
NormalValue    159610
Name: isDefault, dtype: int64
------------------------------------------------------------
NormalValue       794259
ExceptionValue      5741
Name: interestRate_outliers, dtype: int64
interestRate_outliers
ExceptionValue      2916
NormalValue       156694
Name: isDefault, dtype: int64
------------------------------------------------------------
NormalValue       792046
ExceptionValue      7954
Name: installment_outliers, dtype: int64
installment_outliers
ExceptionValue      2152
NormalValue       157458
Name: isDefault, dtype: int64


In [24]:
# Filter the exception values
for fea in numerical_fea:
    data_train = data_train[data_train[fea+'_outliers']=='NormalValue']
    data_train = data_train.reset_index(drop=True) 

### Feature binning

In [25]:
# Feature binning
data['loanAmnt_bin1'] = np.floor_divide(data['loanAmnt'], 1000)
data['loanAmnt_bin2'] = np.floor(np.log10(data['loanAmnt']))
data['loanAmnt_bin3'] = pd.qcut(data['loanAmnt'], 10, labels=False)

In [26]:
for col in ['grade', 'subGrade']: 
    temp_dict = data_train.groupby([col])['isDefault'].agg(['mean']).reset_index().rename(columns={'mean': col + '_target_mean'})
    temp_dict.index = temp_dict[col].values
    temp_dict = temp_dict[col + '_target_mean'].to_dict()
    data_train[col + '_target_mean'] = data_train[col].map(temp_dict)
    data_test_a[col + '_target_mean'] = data_test_a[col].map(temp_dict)

### Feature interaction

In [27]:
# Mean and std
for df in [data_train, data_test_a]:
    for item in ['n0','n1','n2','n4','n5','n6','n7','n8','n9','n10','n11','n12','n13','n14']:
        df['grade_to_mean_' + item] = df['grade'] / df.groupby([item])['grade'].transform('mean')
        df['grade_to_std_' + item] = df['grade'] / df.groupby([item])['grade'].transform('std')

### High dimensional feature encoding

In [28]:
# Label-encoding: subGrade, postCode, title
for col in tqdm(['employmentTitle', 'postCode', 'title','subGrade']):
    le = LabelEncoder()
    le.fit(list(data_train[col].astype(str).values) + list(data_test_a[col].astype(str).values))
    data_train[col] = le.transform(list(data_train[col].astype(str).values))
    data_test_a[col] = le.transform(list(data_test_a[col].astype(str).values))
print('Label Encoding Completed')

100%|██████████| 4/4 [00:04<00:00,  1.17s/it]

Label Encoding Completed


### Remove useless features

In [29]:
# Remove issueDate, id
for data in [data_train, data_test_a]:
    data.drop(['issueDate','id'], axis=1,inplace=True)

In [30]:
features = [f for f in data_train.columns if f not in ['id','issueDate'] and '_outliers' not in f]
fg_data_train = data_train[features]

In [31]:
fg_data_train[:10]

,loanAmnt,term,interestRate,installment,grade,subGrade,employmentTitle,employmentLength,homeOwnership,annualIncome,verificationStatus,isDefault,purpose,postCode,regionCode,dti,delinquency_2years,ficoRangeLow,ficoRangeHigh,openAcc,pubRec,pubRecBankruptcies,revolBal,revolUtil,totalAcc,initialListStatus,applicationType,earliesCreditLine,title,policyCode,n0,n1,n2,n3,n4,n5,n6,n7,n8,n9,n10,n11,n12,n13,n14,issueDateDT,grade_target_mean,subGrade_target_mean,grade_to_mean_n0,grade_to_std_n0,grade_to_mean_n1,grade_to_std_n1,grade_to_mean_n2,grade_to_std_n2,grade_to_mean_n4,grade_to_std_n4,grade_to_mean_n5,grade_to_std_n5,grade_to_mean_n6,grade_to_std_n6,grade_to_mean_n7,grade_to_std_n7,grade_to_mean_n8,grade_to_std_n8,grade_to_mean_n9,grade_to_std_n9,grade_to_mean_n10,grade_to_std_n10,grade_to_mean_n11,grade_to_std_n11,grade_to_mean_n12,grade_to_std_n12,grade_to_mean_n13,grade_to_std_n13,grade_to_mean_n14,grade_to_std_n14
0,35000.0,5,19.52,917.97,5,21,161280,2,2,110000.0,2,1,1,43,32,17.05,0.0,730.0,734.0,7.0,0.0,0.0,24178.0,48.9,27.0,0,0,2001,1,1.0,0.0,2.0,2.0,2.0,4.0,9.0,8.0,4.0,12.0,2.0,7.0,0.0,0.0,0.0,2.0,2587,0.386234,0.380444,1.876011,3.992386,1.874620,4.053876,1.942294,4.023418,1.869160,3.948124,1.897562,4.055665,1.865760,4.017884,1.840872,4.074681,1.851544,4.040923,1.938318,4.024912,1.842210,4.108917,1.852810,4.009823,1.852810,4.009823,1.857394,4.005352,1.856379,3.991791
1,18000.0,5,18.49,461.90,4,16,89538,5,0,46000.0,2,0,0,64,18,27.83,0.0,700.0,704.0,13.0,0.0,0.0,15096.0,38.9,18.0,1,0,2002,5768,1.0,0.0,3.0,5.0,5.0,10.0,7.0,7.0,7.0,13.0,5.0,13.0,0.0,0.0,0.0,2.0,1888,0.304227,0.298190,1.500809,3.193909,1.502905,3.185919,1.504054,3.173189,1.567352,3.204484,1.511316,3.139166,1.515599,3.098975,1.500817,3.139721,1.517874,3.086106,1.504140,3.174194,1.484104,3.173687,1.482248,3.207858,1.482248,3.207858,1.485915,3.204282,1.485103,3.193433
2,12000.0,5,16.99,298.17,4,17,159367,8,0,74000.0,2,0,0,265,14,22.77,0.0,675.0,679.0,11.0,0.0,0.0,4606.0,51.8,27.0,0,0,2006,0,1.0,0.0,0.0,3.0,3.0,0.0,0.0,21.0,4.0,5.0,3.0,11.0,0.0,0.0,0.0,4.0,3044,0.304227,0.302541,1.500809,3.193909,1.360761,2.998190,1.532981,3.241462,1.273891,3.071276,1.162371,3.176718,1.480241,3.125317,1.472698,3.259745,1.406712,3.254085,1.530998,3.244609,1.504230,3.089208,1.482248,3.207858,1.482248,3.207858,1.485915,3.204282,1.315111,3.146801
3,2050.0,3,7.69,63.95,1,3,59830,9,0,35000.0,0,0,0,465,14,17.49,0.0,755.0,759.0,12.0,0.0,0.0,3111.0,8.5,23.0,0,0,2006,0,1.0,0.0,1.0,3.0,3.0,7.0,11.0,3.0,10.0,18.0,3.0,12.0,0.0,0.0,0.0,3.0,2679,0.059838,0.065532,0.375202,0.798477,0.368239,0.796491,0.383245,0.810366,0.380622,0.806605,0.384972,0.802575,0.368526,0.819126,0.369865,0.798404,0.377964,0.799464,0.382750,0.811152,0.370128,0.799459,0.370562,0.801965,0.370562,0.801965,0.371479,0.801070,0.344287,0.793451
4,11500.0,3,14.98,398.54,3,12,85242,1,1,30000.0,2,0,0,3,4,32.60,0.0,665.0,669.0,8.0,1.0,1.0,14021.0,59.7,33.0,1,0,1994,0,1.0,0.0,4.0,4.0,4.0,4.0,16.0,10.0,5.0,21.0,4.0,8.0,0.0,0.0,0.0,2.0,2406,0.224522,0.224686,1.125607,2.395431,1.113406,2.430896,1.133984,2.439745,1.121496,2.368874,1.197930,2.401168,1.120956,2.388727,1.106851,2.450979,1.144817,2.403154,1.133458,2.441340,1.104961,2.446307,1.111686,2.405894,1.111686,2.405894,1.114436,2.403211,1.113827,2.395075
5,12000.0,3,12.99,404.27,3,11,65718,5,2,60000.0,1,1,0,770,13,19.22,0.0,690.0,694.0,15.0,0.0,0.0,27176.0,46.0,21.0,1,0,1994,0,1.0,0.0,7.0,13.0,13.0,7.0,7.0,2.0,13.0,17.0,11.0,15.0,0.0,0.0,0.0,6.0,3257,0.224522,0.204005,1.125607,2.395431,1.085997,2.408741,0.984707,2.361605,1.141867,2.419815,1.133487,2.354374,1.100101,2.459716,1.119411,2.396658,1.136053,2.409156,1.011351,2.376224,1.124941,2.384061,1.111686,2.405894,1.111686,2.405894,1.114436,2.403211,0.923430,2.361914
6,24000.0,3,9.99,774.30,2,7,209276,10,0,150000.0,1,0,2,40,8,5.68,0.0,690.0,694.0,7.0,0.0,0.0,4334.0,68.8,25.0,0,0,1983,18780,1.0,1.0,1.0,3.0,3.0,2.0,7.0,7.0,6.0,17.0,3.0,7.0,0.0,0.0,0.0,2.0,2983,0.131210,0.128111,0.707941,1.635584,0.736477,1.592982,0.766491,1.620731,0.720818,1.6

In [32]:
fg_data_train.to_csv('fg_train_data.csv', sep=',', encoding='utf-8')

### Upload data

To upload the dataset to your own s3 bucket:

* Fill {YOUR_S3_BUCKET} and {YOUR_S3_PATH} with your preferred values in the following cell.
* Uncomment the cell by removing the leading # character.
* Execute the cell.

In [33]:
# !aws s3 cp ./fg_train_data.csv ${MY_S3_BUCKET}/risk/tianchi/